In [ ]:
from ase.build import surface, add_adsorbate, molecule
from ase.spacegroup import crystal
from ase.visualize import view
from ase.io.trajectory import TrajectoryReader
import os
from ase.io import read
import matplotlib.pyplot as plt
import sys
from ase.symbols import string2symbols
import pickle
%matplotlib inline

## Eng to pickle: 
1. Potential energies to a dictionary (O2, N2, and H2 as reference)
2. Turn dictionary to a pickle file
3. Each pickle file corresponds to ref, ads_slab and slab

#### ads_slab and slab, grab lowest energy state from spin paired vs pol


In [ ]:
with open('../pickle_files/HSE_converged_bulk_binding.pkl', 'rb') as handle:
    ads_formation_energy_spin_pair = pickle.load(handle)
handle.close()

sorted_keys = sorted(ads_formation_energy_spin_pair)


#### reference

In [ ]:
def update_reference_eng(reference):
    reference['H'] = 0.5*reference['H2']
    reference['N'] = 0.5*reference['N2']
    reference['O'] = reference['O2'] /2
    return reference

with open('../pickle_files/reference_HSE.pkl', 'rb') as handle:
    reference_dict = pickle.load(handle)
handle.close()

reference_dict = reference_dict['reference']
reference_dict = update_reference_eng(reference_dict)
reference_dict.pop('N2H4')
reference_dict.pop('N2H2')


def get_ref_formation_energy(reference_dict):
    formation_energyies = {} 
    for key in reference_dict:
        if key == 'slab':
            continue
        E0 = reference_dict[key]
        composition = string2symbols(key)
        for atom in composition:
            E0 -= reference_dict[atom]
        E0 = round(E0,3)
        formation_energyies[key] = E0
    return formation_energyies
ref_formation_energy = get_ref_formation_energy(reference_dict)
ref_formation_energy.pop('N')
ref_formation_energy.pop('H')
ref_formation_energy.pop('O')


#### vib energy files: ads_slab and reference

In [ ]:
with open('../pickle_files/converged_bulk_vib.pkl', 'rb') as handle:
    vib = pickle.load(handle)
handle.close()
ads_vib = vib['ads_slab']
slab_vib = vib['slab']
vib_dict = {**slab_vib, **ads_vib}
vib_dict['H'] = []


with open('../pickle_files/reference_vib.pkl', 'rb') as handle:
    vib = pickle.load(handle)
handle.close()
ref_vib = vib['reference']
ref_vib.pop('N2H4')
ref_vib.pop('N2H2')
# ref_vib['H'] = []


### Write to CATMAP compatible text file

In [ ]:
def make_input_file(file_name, sorted_keys,ref_formation_energies,ads_slab_formation_energies,ads_slab_vib_dict,ref_vib_dict):
    #create a header
    header = '\t'.join(['surface_name','site_name',
                        'species_name','formation_energy',
                        'frequencies','reference'])
    lines = [] #list of lines in the output
    for key in ref_formation_energies.keys(): #iterate through keys
        E = ref_formation_energies[key] #raw energy
        site = 'gas'
        surface = 'None'
        frequency = ref_vib_dict[key]
        name = key
        outline = [surface,site,name,E,frequency,'']
        line = '\t'.join([str(w) for w in outline])
        lines.append(line)
    for key in sorted_keys:#ads_slab_formation_energies.keys(): #iterate through keys
        E = ads_slab_formation_energies[key] #raw energy
        site = '152'
        surface = 'slab'
        frequency = ads_slab_vib_dict[key]
        name = key
        outline = [surface,site,name,E,frequency,'']
        line = '\t'.join([str(w) for w in outline])
        lines.append(line)
    lines = [header] + lines
    input_file = '\n'.join(lines)
    input = open(file_name,'w') #open the file name in write mode
    input.write(input_file) #write the text
    input.close() #close the file
    print('Successfully created input file')
    
make_input_file('HSE_catmap_model.txt',sorted_keys,ref_formation_energy,ads_formation_energy,vib_dict,ref_vib)
